import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=480
cls_num=2
btch=20
for i in range (0,cls_num):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=MiddlePhalanxOutlineCorrect(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg0)
print(0,len(vars()['ecg'+str(0)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(0)]))
for i in range (1,cls_num):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (291, 81)
first_test_shape= (600, 81)
classes_quantity= 2
tr_lbls=	 {0, 1}
Count_labels= [166]
max(train_feature_Altitude)= 1.8756
min(train_feature_Altitude)= -1.7195
first_train_sample=
 [ 0.       -0.7065   -0.65997  -0.47379  -0.22616   0.10006   0.39363
  0.69603   0.98459   1.1738    1.4781    1.4308    1.3673    1.4243
  1.5021    1.4319    1.1978    0.98944   0.80072   0.53166   0.23594
 -0.058043 -0.30808  -0.55961  -0.68329  -0.66936  -0.51165  -0.31784
 -0.076799  0.19995   0.44881   0.75216   1.0404    1.3203    1.5863
  1.6022    1.377     1.3586    1.3593    1.3325    1.2374    1.2954
  1.4635    1.3819    1.0603    0.75858   0.44207   0.14705  -0.14366
 -0.3966   -0.58665  -0.49186  -0.50286  -0.62594  -0.72812  -0.88656
 -1.0317   -1.1653   -1.3105   -1.4478   -1.3864   -1.2248   -1.0622
 -0.90622  -0.78891  -0.65143  -0.58693  -0.65616  -0.8165   -0.97518
 -1.1302   -1.2435   -1.2487   -1.2692   -1.3553   -1.332    -1.2132
 -1.1004   -0.95834  -0.795

In [3]:
np.shape(ecg)

(291, 481)

In [4]:
ecg[:,-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
0 125	1 166	
 max = 166


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (0,cls_num):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  0 train shape =  (125, 481)
Up to class  1 train shape =  (291, 481)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[125 166]


In [8]:
xtrain[:,-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (0,cls_num):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (0,cls_num):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 0  >> 
max magnitude class 0  =  2.072583183149495
min magnitude class 0  =  -1.6939263958932065
after normalizing >>
max magnitude class 0  =  1.1016146975829342
min magnitude class 0  =  -0.9816839335302111

 cls 1  >> 
max magnitude class 1  =  1.9305638381922492
min magnitude class 1  =  -1.651088332578463
after normalizing >>
max magnitude class 1  =  1.0230622073444215
min magnitude class 1  =  -0.9579897209468395
each class and its windows =
0 212	1 388	
 max instance in classes of test windows = 388


In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.9816839335302111
max tst = 1.1016146975829342


In [14]:
len(wndws_test)

600

In [15]:
np.shape(wndws_test)

(600, 481)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(166, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (291, 480)
y_train =>  (291,)
X_test  =>  (600, 480)
y_test  =>  (600,)
X_valid  =>  (291, 480)
y_valid  =>  (291,)


In [22]:
y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [23]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [24]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
4/4 [==============================] - 4s 206ms/step - loss: 4.6776 - accuracy: 0.4570 - val_loss: 1.0084 - val_accuracy: 0.5704
Epoch 2/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.8743 - accuracy: 0.5430 - val_loss: 0.9585 - val_accuracy: 0.5704
Epoch 3/1000
4/4 [==============================] - 0s 29ms/step - loss: 0.7329 - accuracy: 0.5464 - val_loss: 0.9514 - val_accuracy: 0.5704
Epoch 4/1000
4/4 [==============================] - 0s 29ms/step - loss: 0.6826 - accuracy: 0.5704 - val_loss: 0.9063 - val_accuracy: 0.4296
Epoch 5/1000
4/4 [==============================] - 0s 31ms/step - loss: 0.6947 - accuracy: 0.4811 - val_loss: 0.8770 - val_accuracy: 0.6632
Epoch 6/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.7135 - accuracy: 0.6289 - val_loss: 0.8360 - val_accuracy: 0.4296
Epoch 7/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.6281 - accuracy: 0.6942 - val_loss: 0.7842 - val_accuracy: 0.5704
Epoch 8/1000

Epoch 59/1000
4/4 [==============================] - 0s 30ms/step - loss: 0.4139 - accuracy: 0.7973 - val_loss: 0.6557 - val_accuracy: 0.7182
Epoch 60/1000
4/4 [==============================] - 0s 32ms/step - loss: 0.3056 - accuracy: 0.8694 - val_loss: 0.6383 - val_accuracy: 0.7732
Epoch 61/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.5306 - accuracy: 0.7766 - val_loss: 0.5973 - val_accuracy: 0.7388
Epoch 62/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.3686 - accuracy: 0.8144 - val_loss: 0.6430 - val_accuracy: 0.7079
Epoch 63/1000
4/4 [==============================] - 0s 30ms/step - loss: 0.3384 - accuracy: 0.8419 - val_loss: 0.6005 - val_accuracy: 0.7595
Epoch 64/1000
4/4 [==============================] - 0s 30ms/step - loss: 0.4636 - accuracy: 0.7801 - val_loss: 0.6392 - val_accuracy: 0.7079
Epoch 65/1000
4/4 [==============================] - 0s 34ms/step - loss: 0.4466 - accuracy: 0.7698 - val_loss: 0.6240 - val_accuracy: 0.7766
Epoch 

4/4 [==============================] - 0s 30ms/step - loss: 0.0912 - accuracy: 0.9656 - val_loss: 0.4349 - val_accuracy: 0.9210
Epoch 117/1000
4/4 [==============================] - 0s 29ms/step - loss: 0.2300 - accuracy: 0.9003 - val_loss: 0.2991 - val_accuracy: 0.8797
Epoch 118/1000
4/4 [==============================] - 0s 29ms/step - loss: 0.2007 - accuracy: 0.9244 - val_loss: 0.3862 - val_accuracy: 0.8110
Epoch 119/1000
4/4 [==============================] - 0s 31ms/step - loss: 0.1663 - accuracy: 0.9210 - val_loss: 0.2041 - val_accuracy: 0.9003
Epoch 120/1000
4/4 [==============================] - 0s 29ms/step - loss: 0.0553 - accuracy: 0.9794 - val_loss: 0.1963 - val_accuracy: 0.9141
Epoch 121/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.0644 - accuracy: 0.9725 - val_loss: 0.4967 - val_accuracy: 0.8591
Epoch 122/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.3044 - accuracy: 0.9038 - val_loss: 0.2356 - val_accuracy: 0.8832
Epoch 123/1000

4/4 [==============================] - 0s 27ms/step - loss: 5.6942e-08 - accuracy: 1.0000 - val_loss: 0.0028 - val_accuracy: 0.9966
Epoch 173/1000
4/4 [==============================] - 0s 31ms/step - loss: 4.8749e-08 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 174/1000
4/4 [==============================] - 0s 30ms/step - loss: 4.3423e-08 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 175/1000
4/4 [==============================] - 0s 31ms/step - loss: 4.0556e-08 - accuracy: 1.0000 - val_loss: 8.3884e-04 - val_accuracy: 1.0000
Epoch 176/1000
4/4 [==============================] - 0s 30ms/step - loss: 3.1543e-08 - accuracy: 1.0000 - val_loss: 7.2430e-04 - val_accuracy: 1.0000
Epoch 177/1000
4/4 [==============================] - 0s 30ms/step - loss: 2.2941e-08 - accuracy: 1.0000 - val_loss: 5.7189e-04 - val_accuracy: 1.0000
Epoch 178/1000
4/4 [==============================] - 0s 31ms/step - loss: 3.0724e-08 - accuracy: 1.0000 - val_loss: 1.12

In [25]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [26]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

19/19 [==============================] - 0s 10ms/step
array([[163,  49],
       [ 74, 314]], dtype=int64)
              precision    recall  f1-score   support

           1       0.69      0.77      0.73       212
           2       0.87      0.81      0.84       388

    accuracy                           0.80       600
   macro avg       0.78      0.79      0.78       600
weighted avg       0.80      0.80      0.80       600



In [27]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
4/4 [==============================] - 2s 226ms/step - loss: 11.6412 - accuracy: 0.4021 - val_loss: 0.9738 - val_accuracy: 0.4296
Epoch 2/1000
4/4 [==============================] - 0s 45ms/step - loss: 1.8844 - accuracy: 0.4502 - val_loss: 0.9979 - val_accuracy: 0.5704
Epoch 3/1000
4/4 [==============================] - 0s 40ms/step - loss: 0.8913 - accuracy: 0.5155 - val_loss: 0.9631 - val_accuracy: 0.5704
Epoch 4/1000
4/4 [==============================] - 0s 40ms/step - loss: 0.6934 - accuracy: 0.5704 - val_loss: 0.8601 - val_accuracy: 0.5704
Epoch 5/1000
4/4 [==============================] - 0s 41ms/step - loss: 0.6883 - accuracy: 0.6564 - val_loss: 0.8921 - val_accuracy: 0.4296
Epoch 6/1000
4/4 [==============================] - 0s 39ms/step - loss: 0.6919 - accuracy: 0.5361 - val_loss: 0.7800 - val_accuracy: 0.5704
Epoch 7/1000
4/4 [==============================] - 0s 40ms/step - loss: 0.7666 - accuracy: 0.6082 - val_loss: 0.7720 - val_accuracy: 0.5704
Epoch 8/100

Epoch 59/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.4015 - accuracy: 0.8247 - val_loss: 0.6109 - val_accuracy: 0.8007
Epoch 60/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.5025 - accuracy: 0.7938 - val_loss: 0.6001 - val_accuracy: 0.7732
Epoch 61/1000
4/4 [==============================] - 0s 44ms/step - loss: 0.4717 - accuracy: 0.7835 - val_loss: 0.6238 - val_accuracy: 0.8144
Epoch 62/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.3877 - accuracy: 0.8179 - val_loss: 0.6115 - val_accuracy: 0.7629
Epoch 63/1000
4/4 [==============================] - 0s 40ms/step - loss: 0.3798 - accuracy: 0.8316 - val_loss: 0.5474 - val_accuracy: 0.7766
Epoch 64/1000
4/4 [==============================] - 0s 40ms/step - loss: 0.6622 - accuracy: 0.7045 - val_loss: 0.6034 - val_accuracy: 0.7216
Epoch 65/1000
4/4 [==============================] - 0s 40ms/step - loss: 0.4983 - accuracy: 0.8247 - val_loss: 0.5731 - val_accuracy: 0.7560
Epoch 

4/4 [==============================] - 0s 37ms/step - loss: 0.2813 - accuracy: 0.8832 - val_loss: 0.2623 - val_accuracy: 0.8832
Epoch 117/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.2160 - accuracy: 0.8969 - val_loss: 0.4287 - val_accuracy: 0.7904
Epoch 118/1000
4/4 [==============================] - 0s 43ms/step - loss: 0.3176 - accuracy: 0.8488 - val_loss: 0.2677 - val_accuracy: 0.9003
Epoch 119/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.1697 - accuracy: 0.9244 - val_loss: 0.2822 - val_accuracy: 0.8763
Epoch 120/1000
4/4 [==============================] - 0s 39ms/step - loss: 0.2366 - accuracy: 0.9072 - val_loss: 0.2491 - val_accuracy: 0.8832
Epoch 121/1000
4/4 [==============================] - 0s 43ms/step - loss: 0.1712 - accuracy: 0.9141 - val_loss: 0.2153 - val_accuracy: 0.9278
Epoch 122/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.3155 - accuracy: 0.8797 - val_loss: 0.4663 - val_accuracy: 0.8076
Epoch 123/1000

4/4 [==============================] - 0s 38ms/step - loss: 0.0768 - accuracy: 0.9897 - val_loss: 0.0447 - val_accuracy: 0.9828
Epoch 174/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.0382 - accuracy: 0.9897 - val_loss: 0.6937 - val_accuracy: 0.8969
Epoch 175/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.3397 - accuracy: 0.8832 - val_loss: 0.1037 - val_accuracy: 0.9622
Epoch 176/1000
4/4 [==============================] - 0s 37ms/step - loss: 0.0873 - accuracy: 0.9656 - val_loss: 0.0955 - val_accuracy: 0.9588
Epoch 177/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.0482 - accuracy: 0.9897 - val_loss: 0.3808 - val_accuracy: 0.9347
Epoch 178/1000
4/4 [==============================] - 0s 43ms/step - loss: 0.0471 - accuracy: 0.9863 - val_loss: 0.0276 - val_accuracy: 0.9966
Epoch 179/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.0458 - accuracy: 0.9863 - val_loss: 0.1001 - val_accuracy: 0.9725
Epoch 180/1000

4/4 [==============================] - 0s 38ms/step - loss: 0.3142 - accuracy: 0.8900 - val_loss: 0.1059 - val_accuracy: 0.9622
Epoch 231/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.4419 - accuracy: 0.8351 - val_loss: 0.2190 - val_accuracy: 0.9313
Epoch 232/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.1786 - accuracy: 0.9450 - val_loss: 0.0885 - val_accuracy: 0.9725
Epoch 233/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.0690 - accuracy: 0.9759 - val_loss: 0.3590 - val_accuracy: 0.8763
Epoch 234/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.0801 - accuracy: 0.9622 - val_loss: 0.0221 - val_accuracy: 0.9931
Epoch 235/1000
4/4 [==============================] - 0s 37ms/step - loss: 0.0135 - accuracy: 1.0000 - val_loss: 0.0381 - val_accuracy: 0.9863
Epoch 236/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.0251 - accuracy: 0.9931 - val_loss: 0.0327 - val_accuracy: 0.9828
Epoch 237/1000

In [28]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

19/19 [==============================] - 0s 10ms/step
array([[153,  59],
       [100, 288]], dtype=int64)
              precision    recall  f1-score   support

           1       0.60      0.72      0.66       212
           2       0.83      0.74      0.78       388

    accuracy                           0.73       600
   macro avg       0.72      0.73      0.72       600
weighted avg       0.75      0.73      0.74       600

